In [7]:
import pandas as pd
import numpy as np

In [8]:
data = pd.read_csv("../data/btc.csv")

# Feature Engineering


In [9]:
import copy
tek_ind_1 = copy.deepcopy(data)
tek_ind_2 = copy.deepcopy(data)

In [10]:
tek_ind_1['daily_return'] = tek_ind_1.close.pct_change().fillna(0) # Percentage change between the current and a prior element
tek_ind_1['cum_daily_return'] = (1 + tek_ind_1['daily_return']).cumprod() # Cummulative Product (+1 is used not so we can ignore the 0s in the first couple rows)

tek_ind_1['H-L'] = tek_ind_1.high - tek_ind_1.low

tek_ind_1['C-O'] = tek_ind_1.close - tek_ind_1.open

tek_ind_1['10day Ma'] = tek_ind_1.close.shift(1).rolling(window = 10).mean().fillna(0)
tek_ind_1['50day Ma'] = tek_ind_1.close.shift(1).rolling(window = 50).mean().fillna(0)
tek_ind_1['200day Ma'] = tek_ind_1.close.shift(1).rolling(window = 200).mean().fillna(0)

import talib
tek_ind_1['rsi'] = talib.RSI(tek_ind_1.close.values, timeperiod = 14)

tek_ind_1['williams %R'] = talib.WILLR(tek_ind_1.high.values,
                                       tek_ind_1.low.values, 
                                      tek_ind_1.close.values, 
                                      14)

# create 7 and 21 days Moving Average
tek_ind_1['ma7'] = tek_ind_1.close.rolling(window=7).mean().fillna(0)
tek_ind_1['ma21'] = tek_ind_1.close.rolling(window=21).mean().fillna(0)

# creating MACD
tek_ind_1['ema_26'] = tek_ind_1.close.ewm(span=26).mean().fillna(0)
tek_ind_1['ema_12'] = tek_ind_1.close.ewm(span=12).mean().fillna(0)
tek_ind_1['macd'] = (tek_ind_1['ema_12'] - tek_ind_1['ema_26'])

In [20]:
# creating bollinger Bands
#Set number of days and standard deviation to use for rolling lookback period for bollinger band calculation
window = 21
no_of_std =2

#calculate rolling mean and standard deviation using number of days set above
rolling_mean = tek_ind_1.close.rolling(window).mean()
rolling_std = tek_ind_1.close.rolling(window).std()

#create two new DF column to hold values of upper and lower Bollinger bands
tek_ind_1['bb_high'] =(rolling_mean + (rolling_std * no_of_std)).fillna(0)
tek_ind_1['bb_low'] =(rolling_mean - (rolling_std * no_of_std)).fillna(0)

#create exponental moving average
tek_ind_1['ema'] = tek_ind_1.close.ewm(com=0.5).mean()

#create momentum
tek_ind_1['momentum'] = tek_ind_1.close - 1

# tek_ind_1.head(20)

In [25]:
# import matplotlib.pyplot as plt
# plt.figure(figsize=(15, 10))
# plt.plot(tek_ind_1['close'], label ='Actual')
# plt.plot(tek_ind_1['bb_high'], label ='10day Ma')
# plt.plot(tek_ind_1['bb_low'], label ='10day Ma')
# plt.legend(loc='best')